In [ ]:
####################################################################################################################
### Deposition efficiency prediction in Cold spray ###
####################################################################################################################
### This script shows the pyhon code (jupyter notebook) used for GUI for applying the ML prediction models described in the manuscript: 
### "Predicting deposition efficiency across diverse cold spray process parameters using machine learning".


# Import libraries
import tkinter as tk
import sys
from PIL import ImageTk, Image
from tkinter import messagebox
from pathlib import Path
from tkinter import filedialog
import pandas as pd
import io
import os
import glob
import csv
from tkinter import ttk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import customtkinter  
from tkinter import messagebox
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import joblib
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.ticker as ticker
import xlsxwriter
from datetime import datetime
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# Initialise lists: parameters, parameter bounds, parameter units
list1 = ['feedstock1_density_kg_per_m3','feedstock_yield_strength_mpa',
                'feedstock1_melting_temperature_celsius','feedstock1_average_particle_size_mycrom','Standoff_distance_mm',
                'Gas_pressure_Mpa','Gas_temperature_celsius','substrate_density_kg_per_m3','substrate_yield_strength_mpa',
                           'substrate_melting_temperature_celsius','surface_travel_speed_mm_per_s','powder_feeder_rate_feedstock1_g_per_min']
list2 = [1000,10,300,11,5,0.5,10,1000,200,300,10,1]
list3 = [15000,2000,3500,49,80,6.9,1100,15000,2000,3500,1000,100]

param_min_max = pd.DataFrame({'parameters': list1, 'Min': list2, 'Max': list3})
warnings.filterwarnings("ignore")

list4 = ["Powder material density", "Powder material yield strength", "Powder material melting temperature",
        "Powder material average particle size", "Nozzle standoff distance", "Gas pressure",
        "Gas temperature", "Substrate material density", "Substrate material yield strength", "Substrate melting temperature",
        "Nozzle surface travel speed","Powder feeder rate"]

units_list =["[kg/m³]","[MPa]", "[\u00b0C]","[\u03BCm]","[mm]","[MPa]","[\u00b0C]","[kg/m³]","[MPa]",
             "[\u00b0C]","[mm/s]","[g/min]"]

# DE prediction function
def button7a_event():
    ## Get values from user input and create dataframe
    feedstock1_morphology = optionmenu2c.get()
    feedstock1_density_kg_per_m3 = float(slider2b_a.get())
    feedstock_yield_strength_mpa = float(slider2b_c.get())
    feedstock1_melting_temperature_celsius = float(slider2b_e.get())
    feedstock1_average_particle_size_mycrom =float(slider2e.get())
    substrate_density_kg_per_m3 = float(slider2d_a.get())
    substrate_yield_strength_mpa = float(slider2d_c.get())
    substrate_melting_temperature_celsius = float(slider2d_e.get())
    substrate_shape = optionmenu2g.get()
    surface_travel_speed_mm_per_s = float(slider3b.get())
    Standoff_distance_mm = float(slider3a.get())
    powder_feeder_rate_feedstock1_g_per_min = float(slider4d.get())
    Gas_type = optionmenu4aa.get()
    Gas_pressure_Mpa = float(slider4a.get())/10
    Gas_temperature_celsius = float(slider4b.get())
    PF_type = optionmenu6b.get()
    RPM  = float(slider4db.get())
    Hopper_vol = float(slider4dbb.get())
    selected_value = radio_var6.get()

    if substrate_shape == "flat":
        substrate_shape__flat = 1
        substrate_shape__rotational = 0
    else:
        substrate_shape__flat = 0
        substrate_shape__rotational = 1

    if feedstock1_morphology == "spherical":
        feedstock1_morphology__spherical = 1
        feedstock1_morphology__irregular = 0
    else:
        feedstock1_morphology__spherical = 0
        feedstock1_morphology__irregular = 1
        
    if Gas_type == "Nitrogen":
        Gas_molecular_weight_u = 28.02
    elif Gas_type == "Helium":
        Gas_molecular_weight_u = 4.002602
    elif Gas_type == "compressed air":
        Gas_molecular_weight_u = 28.96

    if selected_value == "Option6a":
        powder_feeder_rate_feedstock1_g_per_min = (Hopper_vol*(feedstock1_density_kg_per_m3/1000000)* RPM *60)/1000
    elif selected_value == "Option6b":
        powder_feeder_rate_feedstock1_g_per_min=powder_feeder_rate_feedstock1_g_per_min

    column_names = ['feedstock1_density_kg_per_m3','feedstock_yield_strength_mpa',
                'feedstock1_melting_temperature_celsius','feedstock1_average_particle_size_mycrom','Standoff_distance_mm','Gas_molecular_weight_u',
                'Gas_pressure_Mpa','Gas_temperature_celsius','feedstock1_morphology__irregular',
                           'feedstock1_morphology__spherical','substrate_density_kg_per_m3','substrate_yield_strength_mpa',
                           'substrate_melting_temperature_celsius','surface_travel_speed_mm_per_s','powder_feeder_rate_feedstock1_g_per_min'
                          ,'substrate_shape__flat','substrate_shape__rotational']
    
    data_input = [feedstock1_density_kg_per_m3,feedstock_yield_strength_mpa,
                feedstock1_melting_temperature_celsius,feedstock1_average_particle_size_mycrom,Standoff_distance_mm,Gas_molecular_weight_u,
                Gas_pressure_Mpa,Gas_temperature_celsius,feedstock1_morphology__irregular,
                           feedstock1_morphology__spherical,substrate_density_kg_per_m3,substrate_yield_strength_mpa,
                           substrate_melting_temperature_celsius,surface_travel_speed_mm_per_s,powder_feeder_rate_feedstock1_g_per_min
                          ,substrate_shape__flat,substrate_shape__rotational]

    df_data = pd.DataFrame([data_input], columns=column_names)
    
    ## Import scaler and ML prediction model (Neural network)
    scaler = joblib.load('scaler_for_NN_II_20231011.pkl')
    scaled_data = scaler.transform(df_data)
    np.random.seed(26)
    tf.random.set_seed(26)
    model = keras.models.load_model('neural_network_model_II_20231011.h5')
    
    ## Use imported model to make predictions based on user input 
    prediction = model.predict(scaled_data)
    flattened_list = [item for sublist in prediction for item in sublist]
    prediction = flattened_list
    pred_port = round(prediction[0],1)
    
    if pred_port >100:
        pred_port = 100
    else:
        pass
    
    ## Update images in GUI and DE output field
    if pred_port <= 10:
        range_info = 'Expected deviation (real vs. prediction) in range [0-10% DE]:'
        acc_avg = 3.86
        acc_std = 0.95
        output_field7aaa.configure(foreground='orange', font='TkDefaultFont 10 bold')
        output_field7aab.configure(foreground='orange', font='TkDefaultFont 10 bold')
        image_path = "0_10_histogram.png"
        sample_numb = 2
    elif pred_port > 10 and pred_port <= 20:
        range_info = 'Expected deviation (real vs. prediction) in range (10-20% DE]:'
        acc_avg = 11.93
        acc_std = 0.35
        output_field7aaa.configure(foreground='red', font='TkDefaultFont 10 bold')
        output_field7aab.configure(foreground='red', font='TkDefaultFont 10 bold')
        image_path = "10_20_histogram.png"
        sample_numb = 3
    elif pred_port > 20 and pred_port <= 30:
        range_info = 'Expected deviation (real vs. prediction) in range (20-30% DE]:'
        acc_avg = 3.61
        acc_std = 1.83
        output_field7aaa.configure(foreground='orange', font='TkDefaultFont 10 bold')
        output_field7aab.configure(foreground='orange', font='TkDefaultFont 10 bold')
        image_path = "20_30_histogram.png"
        sample_numb = 16
    elif pred_port > 30 and pred_port <= 40:
        range_info = 'Expected deviation (real vs. prediction) in range (30-40% DE]:'
        acc_avg = 4.71
        acc_std = 3.43
        output_field7aaa.configure(foreground='orange', font='TkDefaultFont 10 bold')
        output_field7aab.configure(foreground='orange', font='TkDefaultFont 10 bold')
        image_path = "30_40_histogram.png"
        sample_numb = 11
    elif pred_port > 40 and pred_port <= 50:
        range_info = 'Expected deviation (real vs. prediction) in range (40-50% DE]:'
        acc_avg = 4.63
        acc_std = 2.31
        output_field7aaa.configure(foreground='orange', font='TkDefaultFont 10 bold')
        output_field7aab.configure(foreground='orange', font='TkDefaultFont 10 bold')
        image_path = "40_50_histogram.png"
        sample_numb = 2
    elif pred_port > 50 and pred_port <= 60:
        range_info = 'Expected deviation (real vs. prediction) in range (50-60% DE]:'
        acc_avg = 'NaN'
        acc_std = 'NaN'
        output_field7aaa.configure(foreground='red', font='TkDefaultFont 10 bold')
        output_field7aab.configure(foreground='red', font='TkDefaultFont 10 bold')
        image_path = "50_60_histogram.png"
        sample_numb = 0
    elif pred_port > 60 and pred_port <= 70:
        range_info = 'Expected deviation (real vs. prediction) in range (60-70% DE]:'
        acc_avg = 'NaN'
        acc_std = 'NaN'
        output_field7aaa.configure(foreground='red', font='TkDefaultFont 10 bold')
        output_field7aab.configure(foreground='red', font='TkDefaultFont 10 bold')
        image_path = "60_70_histogram.png"
        sample_numb = 0
    elif pred_port > 70 and pred_port <= 80:
        range_info = 'Expected deviation (real vs. prediction) in range (70-80% DE]:'
        acc_avg = 2.04
        acc_std = 0.61
        output_field7aaa.configure(foreground='orange', font='TkDefaultFont 10 bold')
        output_field7aab.configure(foreground='orange', font='TkDefaultFont 10 bold')
        image_path = "70_80_histogram.png"
        sample_numb = 5
    elif pred_port > 80 and pred_port <= 90:
        range_info = 'Expected deviation (real vs. prediction) in range (80-90% DE]:'
        acc_avg = 3.91
        acc_std = 3.54
        output_field7aaa.configure(foreground='orange', font='TkDefaultFont 10 bold')
        output_field7aab.configure(foreground='orange', font='TkDefaultFont 10 bold')
        image_path = "80_90_histogram.png"
        sample_numb = 14
    elif pred_port > 90 and pred_port <= 100:
        range_info = 'Expected deviation (real vs. prediction) in range (90-100% DE]:'
        acc_avg = 2.43
        acc_std = 1.12
        output_field7aaa.configure(foreground='orange', font='TkDefaultFont 10 bold')
        output_field7aab.configure(foreground='orange', font='TkDefaultFont 10 bold')
        image_path = "90_100_histogram.png"
        sample_numb = 33
        
    output_value7aa.set(range_info)
    output_value7aaa.set(acc_avg)
    output_value7aab.set(acc_std)
    output_value7a.set(pred_port)
    output_value7aabb.set(sample_numb)

    image = Image.open(image_path)
    image = image.resize((250, 150))
    photo = ImageTk.PhotoImage(image)
    image_label.configure(image=photo)
    image_label.image = photo  

    
# Parameter analysis for DE prediction
def button7b_event(ax):
    ## Get values from user input and create dataframe
    Analysis1=optionmenu7c.get()
    Analysis2=optionmenu7e.get()
    feedstock1_morphology = optionmenu2c.get()
    feedstock1_density_kg_per_m3 = float(slider2b_a.get())
    feedstock_yield_strength_mpa = float(slider2b_c.get())
    feedstock1_melting_temperature_celsius = float(slider2b_e.get())
    feedstock1_average_particle_size_mycrom =float(slider2e.get())
    substrate_density_kg_per_m3 = float(slider2d_a.get())
    substrate_yield_strength_mpa = float(slider2d_c.get())
    substrate_melting_temperature_celsius = float(slider2d_e.get())
    substrate_shape = optionmenu2g.get()
    surface_travel_speed_mm_per_s = float(slider3b.get())
    Standoff_distance_mm = float(slider3a.get())
    powder_feeder_rate_feedstock1_g_per_min = float(slider4d.get())
    Gas_type = optionmenu4aa.get()
    Gas_pressure_Mpa = float(slider4a.get())/10
    Gas_temperature_celsius = float(slider4b.get())
    PF_type = optionmenu6b.get()
    RPM  = float(slider4db.get())
    Hopper_vol = float(slider4dbb.get())
    selected_value = radio_var6.get()

    if substrate_shape == "flat":
        substrate_shape__flat = 1
        substrate_shape__rotational = 0
    else:
        substrate_shape__flat = 0
        substrate_shape__rotational = 1

    if feedstock1_morphology == "spherical":
        feedstock1_morphology__spherical = 1
        feedstock1_morphology__irregular = 0
    else:
        feedstock1_morphology__spherical = 0
        feedstock1_morphology__irregular = 1

    if Gas_type == "Nitrogen":
        Gas_molecular_weight_u = 28.02
    elif Gas_type == "Helium":
        Gas_molecular_weight_u = 4.002602
    elif Gas_type == "compressed air":
        Gas_molecular_weight_u = 28.96

    if selected_value == "Option6a":
        powder_feeder_rate_feedstock1_g_per_min = (Hopper_vol*(feedstock1_density_kg_per_m3/1000000)* RPM *60)/1000
    elif selected_value == "Option6b":
        powder_feeder_rate_feedstock1_g_per_min=powder_feeder_rate_feedstock1_g_per_min

    column_names = ['feedstock1_density_kg_per_m3',
                    'feedstock_yield_strength_mpa',
                'feedstock1_melting_temperature_celsius',
                    'feedstock1_average_particle_size_mycrom',
                    'Standoff_distance_mm',
                'Gas_pressure_Mpa',
                    'Gas_temperature_celsius',
                    'substrate_density_kg_per_m3',
                    'substrate_yield_strength_mpa',
                           'substrate_melting_temperature_celsius',
                    'surface_travel_speed_mm_per_s',
                    'powder_feeder_rate_feedstock1_g_per_min',
                   'Gas_molecular_weight_u',
                    'feedstock1_morphology__irregular',
                           'feedstock1_morphology__spherical',
                    'substrate_shape__flat',
                    'substrate_shape__rotational']
    
    data_input = [feedstock1_density_kg_per_m3,
                    feedstock_yield_strength_mpa,
                feedstock1_melting_temperature_celsius,
                    feedstock1_average_particle_size_mycrom,
                    Standoff_distance_mm,
                Gas_pressure_Mpa,
                  Gas_temperature_celsius,
                    substrate_density_kg_per_m3,
                    substrate_yield_strength_mpa,
                           substrate_melting_temperature_celsius,
                    surface_travel_speed_mm_per_s,
                    powder_feeder_rate_feedstock1_g_per_min,
                   Gas_molecular_weight_u,
                    feedstock1_morphology__irregular,
                           feedstock1_morphology__spherical,
                    substrate_shape__flat,
                    substrate_shape__rotational]

    df_data = pd.DataFrame([data_input], columns=column_names)

    df_reordered1 = df_data[['feedstock1_density_kg_per_m3','feedstock_yield_strength_mpa',
                'feedstock1_melting_temperature_celsius','feedstock1_average_particle_size_mycrom','Standoff_distance_mm','Gas_molecular_weight_u',
                'Gas_pressure_Mpa','Gas_temperature_celsius','feedstock1_morphology__irregular',
                           'feedstock1_morphology__spherical','substrate_density_kg_per_m3','substrate_yield_strength_mpa',
                           'substrate_melting_temperature_celsius','surface_travel_speed_mm_per_s','powder_feeder_rate_feedstock1_g_per_min'
                          ,'substrate_shape__flat','substrate_shape__rotational']]

    ## Import scaler and ML prediction model (Neural network)
    scaler = joblib.load('scaler_for_NN_II_20231011.pkl')
    scaled_data = scaler.transform(df_reordered1)
    np.random.seed(26)
    tf.random.set_seed(26)
    model = keras.models.load_model('neural_network_model_II_20231011.h5')
    
    ## Use imported model to make predictions based on user input 
    prediction = model.predict(scaled_data)
    flattened_list = [item for sublist in prediction for item in sublist]
    prediction = flattened_list
    pred_port = round(prediction[0],1)
    if pred_port >100:
        pred_port = 100
    else:
        pass

    ## Analysis for selected two parameters -> create parameter grid for selected parameters
    list_curr_min=list2
    list_curr_max=list3
    position_a1 = list4.index(Analysis1)
    position_a2 = list4.index(Analysis2)

    Min_a1 = list_curr_min[position_a1]
    Max_a1 = list_curr_max[position_a1]
    Min_a2 = list_curr_min[position_a2]
    Max_a2 = list_curr_max[position_a2]
    
    list_a1_a= np.linspace(Min_a1, Max_a1, 50)
    list_a2_a= np.linspace(Min_a2, Max_a2, 50) 
    
    list_a1 = list_a1_a.tolist()
    list_a2 = list_a2_a.tolist()
    
    combinations = []
    for item1 in list_a1:
        for item2 in list_a2:
            combination = (item1, item2)
            combinations.append(combination)
    listA1 = [pair[0] for pair in combinations]
    listA2 = [pair[1] for pair in combinations]

    index_list0= [x for x in range(len(data_input)) if x != position_a1 and x != position_a2]

    rep_list_total=[]
    for i in range (0,len(index_list0)):
        rep_list1 = [data_input[index_list0[i]]] * 2500
        rep_list_total.append(rep_list1)
    list_substrate_shape__flat=[substrate_shape__flat]*2500
    list_substrate_shape__rotational=[substrate_shape__rotational]*2500
    list_feedstock1_morphology__spherical =[feedstock1_morphology__spherical]*2500
    list_feedstock1_morphology__irregular =[feedstock1_morphology__irregular]*2500
    list_Gas_molecular_weight_u = [Gas_molecular_weight_u]*2500
    
    data_all1 = {column_names[index_list0[0]]: rep_list_total[0],
                column_names[index_list0[1]]: rep_list_total[1],
                column_names[index_list0[2]]: rep_list_total[2],
                column_names[index_list0[3]]: rep_list_total[3],
                column_names[index_list0[4]]: rep_list_total[4],
                column_names[index_list0[5]]: rep_list_total[5],
                column_names[index_list0[6]]: rep_list_total[6],
                column_names[index_list0[7]]: rep_list_total[7],
                column_names[index_list0[8]]: rep_list_total[8],
                 column_names[index_list0[9]]: rep_list_total[9]
               }

    data_all2 = {column_names[position_a1]: listA1,
                column_names[position_a2]: listA2            
               }
    
    data_all3 = {'substrate_shape__flat': list_substrate_shape__flat,
                 'substrate_shape__rotational': list_substrate_shape__rotational,
                 'feedstock1_morphology__spherical': list_feedstock1_morphology__spherical,
                 'feedstock1_morphology__irregular':list_feedstock1_morphology__irregular,
                 'Gas_molecular_weight_u':list_Gas_molecular_weight_u}
    
    df1 = pd.DataFrame(data_all1)
    df2 = pd.DataFrame(data_all2)
    df3 = pd.DataFrame(data_all3)
    df_1_3 = pd.concat([df1, df3], axis=1)

    if position_a1 > position_a2:
        df_1_3.insert(position_a1-1, column_names[position_a1], listA1)
        df_1_3.insert(position_a2, column_names[position_a2], listA2)
    
    elif position_a1 < position_a2:
        df_1_3.insert(position_a1, column_names[position_a1], listA1)
        df_1_3.insert(position_a2, column_names[position_a2], listA2)
        
    df_unit = pd.DataFrame({'parameter': list4, 'unit': units_list})
    unit1 = df_unit.loc[df_unit['parameter'] == Analysis1, 'unit'].values[0]
    unit2 = df_unit.loc[df_unit['parameter'] == Analysis2, 'unit'].values[0]

    df_data_now= df_1_3
    df_reordered2 = df_data_now[['feedstock1_density_kg_per_m3','feedstock_yield_strength_mpa',
                'feedstock1_melting_temperature_celsius','feedstock1_average_particle_size_mycrom','Standoff_distance_mm','Gas_molecular_weight_u',
                'Gas_pressure_Mpa','Gas_temperature_celsius','feedstock1_morphology__irregular',
                           'feedstock1_morphology__spherical','substrate_density_kg_per_m3','substrate_yield_strength_mpa',
                           'substrate_melting_temperature_celsius','surface_travel_speed_mm_per_s','powder_feeder_rate_feedstock1_g_per_min'
                          ,'substrate_shape__flat','substrate_shape__rotational']]

    ## Make predictions based on parameter grid to analyse selected parameters
    scaled_data = scaler.transform(df_reordered2)
    np.random.seed(26)
    tf.random.set_seed(26)
    prediction2 = model.predict(scaled_data)
    prediction2=prediction2.tolist()
    flattened_list = [item for sublist in prediction2 for item in sublist]
    prediction2 = flattened_list
    replace_values_greater_than_100(prediction2)
    rounded_values = [round(x, 2) if x >= 0 else 0 for x in prediction2]
    update_figure(ax, list_a2, list_a1,df_data,position_a1,position_a2,pred_port,rounded_values,Analysis1,Analysis2, unit1,unit2)

# Update figures for Parameter analysis    
def update_figure(ax, list_a2, list_a1,df_data,position_a1,position_a2,pred_port,rounded_values,Analysis1,Analysis2,unit1,unit2): 
    global colorbar_created,colorbar

    X, Y = np.meshgrid(list_a2, list_a1)
    z = rounded_values
    x = np.array(X)
    y = np.array(Y)
    z = np.array(z)
    Z = z.reshape(x.shape)

    vmax = float(slider7f.get())
    step = float(slider7g.get())
    vmin = 0
    
    ax.clear()
    contourf_ = ax.contourf(x, y, Z, cmap='viridis', levels=np.arange(vmin, vmax+step, step), extend='max')
    if not colorbar_created:
        if vmax + step < 100:
            levels = np.arange(vmin, vmax + step, step)
        else:
            levels = np.arange(vmin, 100, step)
        tick_positions = np.linspace(vmin, vmax, 10)     
        colorbar = fig.colorbar(contourf_, ax=ax, ticks=tick_positions)
        colorbar.set_label('DE [%]')
        colorbar_created = True
    else:
        colorbar.remove()
        if vmax + step < 100:
            levels = np.arange(vmin, vmax + step, step)
        else:
            levels = np.arange(vmin, 100, step)
        tick_positions = np.linspace(vmin, vmax, 10)
        colorbar = fig.colorbar(contourf_, ax=ax, ticks=tick_positions)
        colorbar.set_label('DE [%]')
    colorbar.ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))

    point_x = round(df_data.iloc[0, position_a2], 1)
    point_y = round(df_data.iloc[0, position_a1], 1)
    pred_port = round(pred_port, 1)
    ax.plot(point_x, point_y, 'ro')
    
    offset = 0.5
    label_point = "({:.1f}/{:.1f}/{:.1f})".format(round(point_x,1),round(point_y,1),round(pred_port,1))
    ax.text(point_x + offset, point_y, label_point, ha='left', va='center', color='red', fontsize=12, fontweight='bold')
    ax.set_xlabel('{} {}'.format(Analysis2, unit2))
    ax.set_ylabel('{} {}'.format(Analysis1, unit1))
    ax.set_title('Parameter-Analysis')
    canvas.draw()
    
# Define widget functions for Tkinter  GUI
def create_empty_graph():
    # Create an empty graph
    fig = Figure(figsize=(6, 4))
    ax = fig.add_subplot(111)
    ax.set_xlabel('Analysis 2')
    ax.set_ylabel('Analysis 1')
    ax.set_title('Parameter Analysis')
    return fig, ax

def slider_event2b_a(value):
    value2b_a = slider2b_a.get()
    value2b_a= round(value2b_a, 0)
    output_value2b_a.set(value2b_a)
    
def slider_event2b_c(value):
    value2b_c = slider2b_c.get()
    value2b_c= round(value2b_c, 0)
    output_value2b_c.set(value2b_c)
    
def slider_event2b_e(value):
    value2b_e = slider2b_e.get()
    value2b_e= round(value2b_e, 0)
    output_value2b_e.set(value2b_e)

def slider_event2d_a(value):
    value2d_a = slider2d_a.get()
    value2d_a= round(value2d_a, 0)
    output_value2d_a.set(value2d_a)
    
def slider_event2d_c(value):
    value2d_c = slider2d_c.get()
    value2d_c= round(value2d_c, 0)
    output_value2d_c.set(value2d_c)
    
def slider_event2d_e(value):
    value2d_e = slider2d_e.get()
    value2d_e= round(value2d_e, 0)
    output_value2d_e.set(value2d_e)    

def slider_event2e(value):
    value2e = slider2e.get()
    value2e= round(value2e, 0)
    output_value2e.set(value2e)
    
def slider_event2i(value):
    value2i = slider2i.get()
    value2i= round(value2i, 1)
    output_value2i.set(value2i)
    
def slider_event3a(value):
    value3a = slider3a.get()
    value3a= round(value3a, 1)
    output_value3a.set(value3a)
    
def slider_event3b(value):
    value3b = slider3b.get()
    value3b= round(value3b, 1)
    output_value3b.set(value3b)
    
def slider_event3c(value):
    value3c = slider3c.get()
    value3c= round(value3c, 0)
    output_value3c.set(value3c)
    
def slider_event1b(value):
    value1b = slider1b.get()
    value1b= round(value1b, 0)
    output_value1b.set(value1b)
    
def slider_event4a(value):
    value4a = slider4a.get()
    value4a= round(value4a, 1)
    output_value4a.set(value4a)
    
def slider_event4b(value):
    value4b = slider4b.get()
    value4b= round(value4b, 1)
    output_value4b.set(value4b)
    
def slider_event4c(value):
    value4c = slider4c.get()
    value4c= round(value4c, 1)
    output_value4c.set(value4c)
    
def slider_event4d(value):
    value4d = slider4d.get()
    value4d= round(value4d, 1)
    output_value4d.set(value4d)
    
def slider_event4db(value):
    value4db = slider4db.get()
    value4db= round(value4db, 1)
    output_value4db.set(value4db)
    
def slider_event4dbb(value):
    value4dbb = slider4dbb.get()
    value4dbb= round(value4dbb, 0)
    output_value4dbb.set(value4dbb)
    
def slider_event5b(value):
    value5b = slider5b.get()
    value5b= round(value5b, 4)
    output_value5b.set(value5b)
    
def slider_event5e(value):
    value5e = slider5e.get()
    value5e= round(value5e, 1)
    output_value5e.set(value5e)
    
def slider_event7f(value):
    value7f = slider7f.get()
    value7f= round(value7f, 0)
    output_value7f.set(value7f)
    
def slider_event7g(value):
    value7g = slider7g.get()
    value7g= round(value7g, 1)
    output_value7g.set(value7g)

def replace_values_greater_than_100(lst):
    for i in range(len(lst)):
        if lst[i] > 100:
            lst[i] = 100

def button1_event():
    extra_val=int(slider1b.get())/100
    factor1 = 1-extra_val
    factor2 = 1+extra_val
    for i in range (0, len(list1)):
        value1= (param_min_max.loc[param_min_max['parameters'] == list1[i], 'Min'].values[0])*factor1
        value2= (param_min_max.loc[param_min_max['parameters'] == list1[i], 'Max'].values[0])*factor2
        slider_list[i].configure(from_=value1, to = value2)
    
def button5_event():
    messagebox.showinfo("Information", "Heat treatment (HT) is conducted in a vacuum oven at the chosen vacuum level. The maximum HT temperature is 1100\u00b0C. The sample is cooled down in the vacuum oven between the maximum HT temperature and recrystallization temperature of Titanium. HT time is selected to achieve through thicknss HT.")

def button5b_event():
    messagebox.showinfo("Information", "Hot isostatic pressing (HIP) treatment is conducted at the chosen temperature. The pressure is 100 MPa. The sample is cooled down in the vacuum oven between the maximum HT temperature and recrystallization temperature of Titanium. HIP time is selected to achieve through thickness HIP.")

def optionmenu_callback2b(choice):
    if optionmenu2b.get() == "Titanium":
        slider2b_a.configure(state="disabled",button_color='gray88')
        output_value2b_a.set("4500")
        slider2b_a.set(4500)
        slider2b_c.configure(state="disabled",button_color='gray88')
        output_value2b_c.set("240")
        slider2b_c.set(240)
        slider2b_e.configure(state="disabled",button_color='gray88')
        output_value2b_e.set("1670")
        slider2b_e.set(1670)
    elif optionmenu2b.get() == "Ti-6Al-4V":
        slider2b_a.configure(state="disabled",button_color='gray88')
        output_value2b_a.set("4400")
        slider2b_a.set(4400)
        slider2b_c.configure(state="disabled",button_color='gray88')
        output_value2b_c.set("880")
        slider2b_c.set(880)
        slider2b_e.configure(state="disabled",button_color='gray88')
        output_value2b_e.set("1600")
        slider2b_e.set(1600)
    elif optionmenu2b.get() == "Copper":
        slider2b_a.configure(state="disabled",button_color='gray88')
        output_value2b_a.set("8980")
        slider2b_a.set(8980)
        slider2b_c.configure(state="disabled",button_color='gray88')
        output_value2b_c.set("140")
        slider2b_c.set(140)
        slider2b_e.configure(state="disabled",button_color='gray88')
        output_value2b_e.set("1085")
        slider2b_e.set(1085)
    elif optionmenu2b.get() == "Invar 36":
        slider2b_a.configure(state="disabled",button_color='gray88')
        output_value2b_a.set("8100")
        slider2b_a.set(8100)
        slider2b_c.configure(state="disabled",button_color='gray88')
        output_value2b_c.set("230")
        slider2b_c.set(230)
        slider2b_e.configure(state="disabled",button_color='gray88')
        output_value2b_e.set("1430")
        slider2b_e.set(1430)
    elif optionmenu2b.get() == "Custom":
        slider2b_a.configure(state="normal",button_color='DodgerBlue3')
        output_value2b_a.set("9999")
        slider2b_a.set(9999)
        slider2b_c.configure(state="normal",button_color='DodgerBlue3')
        output_value2b_c.set("999")
        slider2b_c.set(999)
        slider2b_e.configure(state="normal",button_color='DodgerBlue3')
        output_value2b_e.set("999")
        slider2b_e.set(999)
        
def optionmenu_callback2d(choice):
    if optionmenu2d.get() == "Titanium":
        slider2d_a.configure(state="disabled",button_color='gray88')
        output_value2d_a.set("4500")
        slider2d_a.set(4500)
        slider2d_c.configure(state="disabled",button_color='gray88')
        output_value2d_c.set("240")
        slider2d_c.set(240)
        slider2d_e.configure(state="disabled",button_color='gray88')
        output_value2d_e.set("1670")
        slider2d_e.set(1670)        
    elif optionmenu2d.get() == "Aluminium":
        slider2d_a.configure(state="disabled",button_color='gray88')
        output_value2d_a.set("2700")
        slider2d_a.set(2700)
        slider2d_c.configure(state="disabled",button_color='gray88')
        output_value2d_c.set("90")
        slider2d_c.set(90)
        slider2d_e.configure(state="disabled",button_color='gray88')
        output_value2d_e.set("660")
        slider2d_e.set(660)
    elif optionmenu2d.get() == "Copper":
        slider2d_a.configure(state="disabled",button_color='gray88')
        output_value2d_a.set("8980")
        slider2d_a.set(8980)
        slider2d_c.configure(state="disabled",button_color='gray88')
        output_value2d_c.set("140")
        slider2d_c.set(140)
        slider2d_e.configure(state="disabled",button_color='gray88')
        output_value2d_e.set("1085")
        slider2d_e.set(1085)
    elif optionmenu2d.get() == "Custom":
        slider2d_a.configure(state="normal",button_color='DodgerBlue3')
        output_value2d_a.set("9999")
        slider2d_a.set(9999)
        slider2d_c.configure(state="normal",button_color='DodgerBlue3')
        output_value2d_c.set("999")
        slider2d_c.set(999)
        slider2d_e.configure(state="normal",button_color='DodgerBlue3')
        output_value2d_e.set("999")
        slider2d_e.set(999)
        
def optionmenu_callback6b(choice):
    if optionmenu6b.get() == "Impact Powder feeder 5/11":
        slider4dbb.configure(state="disabled",button_color='gray88')
        output_value4dbb.set("3700")
        slider4dbb.set(3700)
    elif optionmenu6b.get() == "custom":
        slider4dbb.configure(state="normal",button_color='DodgerBlue3')
        output_value4dbb.set("3000")
        slider4dbb.set(3000)
    
def update_slider_state():
    selected_value = radio_var6.get()
    if selected_value == "Option6a":
        slider4d.configure(state="disabled",button_color='gray88')
        output_value4d.set("-")
        slider4db.configure(state="normal", button_color='DodgerBlue3')
        output_value4db.set(value4db)
        slider4dbb.configure(state="normal", button_color='DodgerBlue3')
        output_value4dbb.set(value4dbb)
        optionmenu6b.configure(state="normal",fg_color='gray99',button_color='gray79',
                                           button_hover_color='DodgerBlue3')
    else:
        slider4d.configure(state="normal",button_color='DodgerBlue3')
        output_value4d.set(value4d)
        slider4db.configure(state="disabled", button_color='gray88')
        output_value4db.set("-")
        slider4dbb.configure(state="disabled", button_color='gray88')
        output_value4dbb.set("-")
        optionmenu6b.configure(state="disabled",fg_color='gray99',button_color='gray79',
                                           button_hover_color='gray85')
def update_slider_state2():
    selected_value = radio_var6.get()
    if selected_value == "Option6b":
        slider4d.configure(state="normal",button_color='DodgerBlue3')
        output_value4d.set(value4d)
        slider4db.configure(state="disabled", button_color='gray88')
        output_value4db.set("-")
        slider4dbb.configure(state="disabled", button_color='gray88')
        output_value4dbb.set("-")
        optionmenu6b.configure(state="disabled",fg_color='gray99',button_color='gray79',
                                           button_hover_color='gray85')
    else:
        slider4d.configure(state="disabled",button_color='gray88')
        output_value4d.set("-")
        slider4db.configure(state="normal", button_color='DodgerBlue3')
        output_value4db.set(value4db)
        slider4dbb.configure(state="normal", button_color='DodgerBlue3')
        output_value4dbb.set(value4dbb)
        optionmenu6b.configure(state="normal",fg_color='gray99',button_color='gray79',
                                           button_hover_color='DodgerBlue3')
        
root = tk.Tk()
root.title("Machine learning")
root.geometry("1366x1080+0+0")
root.minsize(1366, 1080)
root.maxsize(1366, 1080)
notebook = ttk.Notebook(root)
slider_list =[]
tab1 = ttk.Frame(notebook)
notebook.add(tab1, text="Deposition efficiency")

## First frame
first_frame = tk.Frame(tab1, width=730, height=120, highlightbackground='black', highlightthickness=1)
first_frame.place(x=50, y=10)

label1a = tk.Label(first_frame, text="Model settings", font=("Arial", 15, "bold"))
label1a.place(x=200, y=5)

label_1d = tk.Label(first_frame, text="ML algorithm")
label_1d.place(x=10, y=40)

output_value1d = tk.StringVar(value = "Neural network")  # Create StringVar() variable
output_field1d = tk.Entry(first_frame, textvariable=output_value1d, state='readonly')
output_field1d.place(x=130, y=40)

label_1e = tk.Label(first_frame, text="Training dataset")
label_1e.place(x=10, y=90)

output_value1e = tk.StringVar(value = "TTT_DE_data")  # Create StringVar() variable
output_field1e = tk.Entry(first_frame, textvariable=output_value1e, state='readonly', width=50)
output_field1e.place(x=130, y=90)

label_1f = tk.Label(first_frame, text="Training date")
label_1f.place(x=480, y=90)

output_value1f = tk.StringVar(value = "01/10/2023")  # Create StringVar() variable
output_field1f = tk.Entry(first_frame, textvariable=output_value1f, state='readonly')
output_field1f.place(x=580, y=90)

## Second frame 
second_frame = tk.Frame(tab1, width=730, height=310, highlightbackground='black', highlightthickness=1)
second_frame.place(x=50, y=135)

label_2nd = tk.Label(second_frame, text="Spray specification",font=("Arial", 15, "bold"))
label_2nd.place(x=200, y=0)

second_frame_b = tk.Frame(second_frame, width=350, height=130, highlightbackground='black', highlightthickness=1)
second_frame_b.place(x=370, y=35)

second_frame_c = tk.Frame(second_frame, width=350, height=130, highlightbackground='black', highlightthickness=1)
second_frame_c.place(x=370, y=175)

### Powder material
label2b = tk.Label(second_frame, text="Powder material")
label2b.place(x=10, y=40)

options2b = ["Titanium", "Ti-6Al-4V","Copper", "Invar 36", "Custom"]
optionmenu2b = customtkinter.CTkOptionMenu(second_frame, values=options2b,command=optionmenu_callback2b,fg_color='gray99',button_color='gray79',
                                           button_hover_color='DodgerBlue3')
optionmenu2b.set("Custom")
optionmenu2b.place(x=200, y=35)

### Density
label2b_a = tk.Label(second_frame_b, text="M.Density")
label2b_a.place(x=10, y=5)

slider2b_a= customtkinter.CTkSlider(master=second_frame_b, from_=1000, to=15000, width= 70, command=slider_event2b_a, button_color='DodgerBlue3')
slider2b_a.place(x=100,y=5)

value2b_a = slider2b_a.get()
value2b_a= round(value2b_a, 0)

output_value2b_a = tk.StringVar(value = value2b_a) # Create StringVar() variable
output_field2b_a = tk.Entry(second_frame_b, textvariable=output_value2b_a, state='readonly', width = 7)
output_field2b_a.place(x=220,y=5)

label2b_b = tk.Label(second_frame_b, text="g/m³")
label2b_b.place(x=270,y=5)

### Yield strength
label2b_c = tk.Label(second_frame_b, text="M.Yield strength")
label2b_c.place(x=10, y=45)

slider2b_c= customtkinter.CTkSlider(master=second_frame_b, from_=10, to=2000, width= 70, command=slider_event2b_c, button_color='DodgerBlue3')
slider2b_c.place(x=100,y=45)

value2b_c = slider2b_c.get()
value2b_c= round(value2b_c, 0)

output_value2b_c = tk.StringVar(value = value2b_c) # Create StringVar() variable
output_field2b_c = tk.Entry(second_frame_b, textvariable=output_value2b_c, state='readonly', width = 7)
output_field2b_c.place(x=220,y=45)

label2b_d = tk.Label(second_frame_b, text="MPa")
label2b_d.place(x=270,y=45)

### Melting point
label2b_e = tk.Label(second_frame_b, text="M.Melting point")
label2b_e.place(x=10, y=85)

slider2b_e= customtkinter.CTkSlider(master=second_frame_b, from_=300, to=3500, width= 70, command=slider_event2b_e, button_color='DodgerBlue3')
slider2b_e.place(x=100,y=85)

value2b_e = slider2b_e.get()
value2b_e= round(value2b_e, 0)

output_value2b_e = tk.StringVar(value = value2b_e) # Create StringVar() variable
output_field2b_e = tk.Entry(second_frame_b, textvariable=output_value2b_e, state='readonly', width = 7)
output_field2b_e.place(x=220,y=85)
''
label2b_f = tk.Label(second_frame_b, text="\u00b0C")
label2b_f.place(x=270,y=85)

### Particle size
label2d = tk.Label(second_frame, text="Average powder size")
label2d.place(x=10, y=90)

slider2e= customtkinter.CTkSlider(master=second_frame, from_=11, to=49, width= 70, command=slider_event2e, button_color='DodgerBlue3')
slider2e.place(x=200,y=85)

value2e = slider2e.get()
value2e= round(value2e, 0)

output_value2e = tk.StringVar(value = value2e) # Create StringVar() variable
output_field2e = tk.Entry(second_frame, textvariable=output_value2e, state='readonly', width = 5)
output_field2e.place(x=270,y=90)

label2e = tk.Label(second_frame, text="\u03BCm")
label2e.place(x=300,y=90)

### Powder morphology

label2c = tk.Label(second_frame, text="Powder morphology")
label2c.place(x=10, y=140)

options2c = ["irregular", "spherical"]
optionmenu2c = customtkinter.CTkOptionMenu(second_frame, values=options2c,fg_color='gray99',button_color='gray79',
                                           button_hover_color='DodgerBlue3')
optionmenu2c.set("irregular")
optionmenu2c.place(x=200, y=135)

### Substrate  material
label2d = tk.Label(second_frame, text="Substrate material")
label2d.place(x=10, y=180)

options2d = ["Titanium","Aluminium", "Copper", "Custom"]
optionmenu2d = customtkinter.CTkOptionMenu(second_frame, values=options2d,command=optionmenu_callback2d,fg_color='gray99',button_color='gray79',
                                           button_hover_color='DodgerBlue3')
optionmenu2d.set("Custom")
optionmenu2d.place(x=200, y=175)

### Spray type  
label2g = tk.Label(second_frame, text="Spray type")
label2g.place(x=10, y=220)

options2g = ["flat","rotational"]
optionmenu2g = customtkinter.CTkOptionMenu(second_frame, values=options2g,fg_color='gray99',button_color='gray79',
                                           button_hover_color='DodgerBlue3')
optionmenu2g.set("flat")
optionmenu2g.place(x=200, y=215)

### Substrate Density
label2d_a = tk.Label(second_frame_c, text="S.Density")
label2d_a.place(x=10, y=5)

slider2d_a= customtkinter.CTkSlider(master=second_frame_c, from_=1000, to=15000, width= 70, command=slider_event2d_a, button_color='DodgerBlue3')
slider2d_a.place(x=100,y=5)

value2d_a = slider2d_a.get()
value2d_a= round(value2d_a, 0)

output_value2d_a = tk.StringVar(value = value2d_a) # Create StringVar() variable
output_field2d_a = tk.Entry(second_frame_c, textvariable=output_value2d_a, state='readonly', width = 7)
output_field2d_a.place(x=220,y=5)

label2d_b = tk.Label(second_frame_c, text="g/m³")
label2d_b.place(x=270,y=5)

### Yield strength
label2d_c = tk.Label(second_frame_c, text="S.Yield strength")
label2d_c.place(x=10, y=45)

slider2d_c= customtkinter.CTkSlider(master=second_frame_c, from_=10, to=2000, width= 70, command=slider_event2d_c, button_color='DodgerBlue3')
slider2d_c.place(x=100,y=45)

value2d_c = slider2d_c.get()
value2d_c= round(value2d_c, 0)

output_value2d_c = tk.StringVar(value = value2d_c) # Create StringVar() variable
output_field2d_c = tk.Entry(second_frame_c, textvariable=output_value2d_c, state='readonly', width = 7)
output_field2d_c.place(x=220,y=45)

label2d_d = tk.Label(second_frame_c, text="MPa")
label2d_d.place(x=270,y=45)

### Melting point
label2d_e = tk.Label(second_frame_c, text="S.Melting point")
label2d_e.place(x=10, y=85)

slider2d_e= customtkinter.CTkSlider(master=second_frame_c, from_=300, to=3500, width= 70, command=slider_event2d_e, button_color='DodgerBlue3')
slider2d_e.place(x=100,y=85)

value2d_e = slider2d_e.get()
value2d_e= round(value2d_e, 0)

output_value2d_e = tk.StringVar(value = value2d_e) # Create StringVar() variable
output_field2d_e = tk.Entry(second_frame_c, textvariable=output_value2d_e, state='readonly', width = 7)
output_field2d_e.place(x=220,y=85)

label2d_f = tk.Label(second_frame_c, text="\u00b0C")
label2d_f.place(x=270,y=85)

## Third frame 
third_frame = tk.Frame(tab1, width=515, height=315, highlightbackground='black', highlightthickness=1)
third_frame.place(x=790, y=315)

label_3rd = tk.Label(third_frame, text="Robot parameters",font=("Arial", 15, "bold"))
label_3rd.place(x=200, y=0)

### Standoff distance 
label3a = tk.Label(third_frame, text="Standoff distance")
label3a.place(x=10, y=40)

slider3a = customtkinter.CTkSlider(master=third_frame, from_=5, to=80,command=slider_event3a, button_color='DodgerBlue3')
slider3a.place(x=200, y=40)

value3a = slider3a.get()
value3a= round(value3a, 1)

output_value3a = tk.StringVar(value=value3a) # Create StringVar() variable
output_field3a = tk.Entry(third_frame, textvariable=output_value3a, state='readonly', width = 5)
output_field3a.place(x=400,y=40)

label3b = tk.Label(third_frame, text="mm")
label3b.place(x=430,y=40)

### Spray angle
label3c = tk.Label(third_frame, text="Spray angle")
label3c.place(x=10, y=90)

slider3c = customtkinter.CTkSlider(master=third_frame, from_=60, to=120,command=slider_event3c,state="disabled",button_color='gray88')
slider3c.place(x=200, y=90)

value3c = slider3c.get()
value3c= round(value3c, 0)

output_value3c = tk.StringVar(value =value3c) # Create StringVar() variable
output_field3c = tk.Entry(third_frame, textvariable=output_value3c, state='readonly', width = 5)
output_field3c.place(x=400,y=90)

label3d = tk.Label(third_frame, text="\u00B0")
label3d.place(x=430,y=90)

### Surface speed
label3b = tk.Label(third_frame, text="Surface speed")
label3b.place(x=10, y=140)

slider3b = customtkinter.CTkSlider(master=third_frame, from_=10, to=1000,command=slider_event3b, button_color='DodgerBlue3')
slider3b.place(x=200, y=140)

value3b = slider3b.get()
value3b= round(value3b, 1)

output_value3b = tk.StringVar(value=value3b) # Create StringVar() variable
output_field3b = tk.Entry(third_frame, textvariable=output_value3b, state='readonly', width = 5)
output_field3b.place(x=400,y=140)

label3b = tk.Label(third_frame, text="mm/s")
label3b.place(x=430,y=140)

## Fourth frame
fourth_frame = tk.Frame(tab1, width=730, height=180, highlightbackground='black', highlightthickness=1)
fourth_frame.place(x=50, y=450)

label_4th = tk.Label(fourth_frame, text="Gas parameters",font=("Arial", 15, "bold"))
label_4th.place(x=140, y=0)

### Gas type
label4aa = tk.Label(fourth_frame, text="Gas type")
label4aa.place(x=10, y=40)

options4aa = ["Nitrogen", "Helium", "compressed air"]
optionmenu4aa = customtkinter.CTkOptionMenu(fourth_frame, values=options4aa,fg_color='gray99',button_color='gray79',
                                           button_hover_color='DodgerBlue3')
optionmenu4aa.set("Nitrogen")
optionmenu4aa.place(x=200, y=35)

### Gas pressure
label4a = tk.Label(fourth_frame, text="Gas pressure")
label4a.place(x=10, y=90)

slider4a = customtkinter.CTkSlider(master=fourth_frame, from_=5, to=69,command=slider_event4a, button_color='DodgerBlue3')
slider4a.place(x=200, y=90)

value4a = slider4a.get()
value4a= round(value4a, 1)

output_value4a = tk.StringVar(value=value4a) # Create StringVar() variable
output_field4a = tk.Entry(fourth_frame, textvariable=output_value4a, state='readonly', width = 5)
output_field4a.place(x=400,y=90)

label4a = tk.Label(fourth_frame, text="bar")
label4a.place(x=430,y=90)

### Gas temperature
label4b = tk.Label(fourth_frame, text="Gas temperature")
label4b.place(x=10, y=140)

slider4b = customtkinter.CTkSlider(master=fourth_frame, from_=300, to=1100,command=slider_event4b, button_color='DodgerBlue3')
slider4b.place(x=200, y=140)

value4b = slider4b.get()
value4b= round(value4b, 1)

output_value4b = tk.StringVar(value=value4b) # Create StringVar() variable
output_field4b = tk.Entry(fourth_frame, textvariable=output_value4b, state='readonly', width = 5)
output_field4b.place(x=400,y=140)

label4b = tk.Label(fourth_frame, text="\u00b0C")
label4b.place(x=430,y=140)

## Sixth frame 
sixth_frame = tk.Frame(tab1, width=515, height=300, highlightbackground='black', highlightthickness=1)
sixth_frame.place(x=790, y=10)

label_6th = tk.Label(sixth_frame, text="Powder feeder",font=("Arial", 15, "bold"))
label_6th.place(x=230, y=0)

label_6a = tk.Label(sixth_frame, text="Select")
label_6a.place(x=10, y=40)

radio_var6 = tk.StringVar()
radio_button6a = ttk.Radiobutton(sixth_frame, text="RPM", variable=radio_var6, value="Option6a",command=update_slider_state)
radio_button6a.place(x=200, y=40)
radio_button6b = ttk.Radiobutton(sixth_frame, text="g/min", variable=radio_var6, value="Option6b",command=update_slider_state2)
radio_button6b.place(x=250, y=40)
radio_var6.set("Option6b")

### Powder feeder rate rate
label4d = tk.Label(sixth_frame, text="Powder feeder rate")
label4d.place(x=10, y=90)

slider4d = customtkinter.CTkSlider(master=sixth_frame, from_=10, to=283,command=slider_event4d, button_color='DodgerBlue3')
slider4d.place(x=200, y=90)

value4d = slider4d.get()
value4d= round(value4d, 1)

output_value4d = tk.StringVar(value=value4d) # Create StringVar() variable
output_field4d = tk.Entry(sixth_frame, textvariable=output_value4d, state='readonly', width = 7)
output_field4d.place(x=400,y=90)

label4d = tk.Label(sixth_frame, text="g/min")
label4d.place(x=450,y=90)

### Powder feeder type
label6a = tk.Label(sixth_frame, text="Powder feeder type")
label6a.place(x=10, y=140)

options6b = ["Impact Powder feeder 5/11", "custom"]
optionmenu6b = customtkinter.CTkOptionMenu(sixth_frame, values=options6b,state="disabled",fg_color='gray99',button_color='gray79',
                                           button_hover_color='gray85')
optionmenu6b.set("custom")
optionmenu6b.place(x=200, y=135)

### Powder feeder rate rate
label4db = tk.Label(sixth_frame, text="Powder feeder rate")
label4db.place(x=10, y=190)

slider4db = customtkinter.CTkSlider(master=sixth_frame, from_=1, to=20,command=slider_event4db, state="disabled",button_color='gray88')
slider4db.place(x=200, y=190)

value4db = slider4db.get()
value4db= round(value4db, 1)

output_value4db = tk.StringVar(value=value4db) # Create StringVar() variable
output_value4db.set("-")
output_field4db = tk.Entry(sixth_frame, textvariable=output_value4db, state='readonly', width = 7)
output_field4db.place(x=400,y=190)

label4dbb = tk.Label(sixth_frame, text="RPM")
label4dbb.place(x=450,y=190)

### Hopper Volume
label4dbb = tk.Label(sixth_frame, text="Hopper Volume")
label4dbb.place(x=10, y=240)

slider4dbb = customtkinter.CTkSlider(master=sixth_frame, from_=1000, to=5000,command=slider_event4dbb, state="disabled",button_color='gray88')
slider4dbb.place(x=200, y=240)

value4dbb = slider4dbb.get()
value4dbb= round(value4dbb, 0)

output_value4dbb = tk.StringVar(value=value4dbb) # Create StringVar() variable
output_value4dbb.set("-")
output_field4dbb = tk.Entry(sixth_frame, textvariable=output_value4dbb, state='readonly', width = 7)
output_field4dbb.place(x=400,y=240)

label4dbbb = tk.Label(sixth_frame, text="ccm")
label4dbbb.place(x=450,y=240)

### Seventh frame 
seventh_frame = tk.Frame(tab1, width=1255, height=320, highlightbackground='black', highlightthickness=1)
seventh_frame.place(x=50, y=645)

label_7th = tk.Label(seventh_frame, text="Output / Model prediction",font=("Arial", 15, "bold"))
label_7th.place(x=550, y=5)

button7a = tk.Button(seventh_frame, text="Predict DE", width=15, height=1,command=button7a_event)
button7a.place(x=10, y=35)

output_value7aa = tk.StringVar(value="Expected deviation (real vs. prediction)") # Create StringVar() variable
output_field7aa = tk.Entry(seventh_frame, textvariable=output_value7aa, state='readonly', width = 57)
output_field7aa.place(x=10,y=90)

label7aaa = tk.Label(seventh_frame, text="Average")
label7aaa.place(x=10, y=130)

output_value7aaa = tk.StringVar(value="3.58") # Create StringVar() variable
output_field7aaa = tk.Entry(seventh_frame, textvariable=output_value7aaa, state='readonly', width = 7)
output_field7aaa.place(x=70,y=130)

label7aaa2 = tk.Label(seventh_frame, text="% DE")
label7aaa2.place(x=100, y=130)

label7aab = tk.Label(seventh_frame, text="Stddev")
label7aab.place(x=210, y=130)

output_value7aab = tk.StringVar(value="2.85") # Create StringVar() variable
output_field7aab = tk.Entry(seventh_frame, textvariable=output_value7aab, state='readonly', width = 7)
output_field7aab.place(x=260,y=130)

label7aab2 = tk.Label(seventh_frame, text="% DE")
label7aab2.place(x=290, y=130)

image_path = "ALL_histogram.png"
image = Image.open(image_path)
image = image.resize((250, 150))
photo = ImageTk.PhotoImage(image)

image_label = tk.Label(seventh_frame, image=photo)
image_label.place(x=10, y=160)

label7aabb = tk.Label(seventh_frame, text="# samples:")
label7aabb.place(x=265, y=170)

output_value7aabb = tk.StringVar(value="86") # Create StringVar() variable
output_field7aabb = tk.Entry(seventh_frame, textvariable=output_value7aabb, state='readonly', width = 5)
output_field7aabb.place(x=265,y=200)

label7aabc = tk.Label(seventh_frame, text="seen/unseen:")
label7aabc.place(x=265, y=230)

output_value7aabc = tk.StringVar(value="80/20") # Create StringVar() variable
output_field7aabc = tk.Entry(seventh_frame, textvariable=output_value7aabc, state='readonly', width = 10)
output_field7aabc.place(x=265,y=260)

output_value7a = tk.StringVar(value=" ") # Create StringVar() variable
output_field7a = tk.Entry(seventh_frame, textvariable=output_value7a, state='readonly', width = 7)
output_field7a.place(x=160,y=40)

label7b = tk.Label(seventh_frame, text="%")
label7b.place(x=200, y=40)

label7a = tk.Label(seventh_frame, text="*Output is estimated based on user selected parameters. Estimations may deviate from real properties",font=("Arial", 5))
label7a.place(x=10, y=10)

button7b = tk.Button(seventh_frame, text="Analyse", width=8, height=1,command=lambda:button7b_event(ax))
button7b.place(x=370, y=240)

label7a = tk.Label(seventh_frame, text="DE prediction software V1.0 was created on 05/10/2023 by Martin Eberle (SEAM/Titomic)",font=("Arial", 5))
label7a.place(x=930, y=300)

label7c = tk.Label(seventh_frame, text="Analyse 1")
label7c.place(x=370, y=40)

options7c = ["Powder material density", "Powder material yield strength", "Powder material melting temperature",
        "Powder material average particle size", "Nozzle standoff distance", "Gas pressure",
        "Gas temperature", "Substrate material density", "Substrate material yield strength", "Substrate melting temperature",
        "Nozzle surface travel speed","Powder feeder rate"]

optionmenu7c = customtkinter.CTkOptionMenu(seventh_frame, values=options7c,fg_color='gray99',button_color='gray79',
                                           button_hover_color='DodgerBlue3')
optionmenu7c.set("Gas pressure")
optionmenu7c.place(x=460, y=35)

label7e = tk.Label(seventh_frame, text="Analyse 2")
label7e.place(x=370, y=90)

options7e = ["Powder material density", "Powder material yield strength", "Powder material melting temperature",
        "Powder material average particle size", "Nozzle standoff distance", "Gas pressure",
        "Gas temperature", "Substrate material density", "Substrate material yield strength", "Substrate melting temperature",
        "Nozzle surface travel speed","Powder feeder rate"]

optionmenu7e = customtkinter.CTkOptionMenu(seventh_frame, values=options7e,fg_color='gray99',button_color='gray79',
                                           button_hover_color='DodgerBlue3')
optionmenu7e.set("Gas temperature")
optionmenu7e.place(x=460, y=90)

### Settings figure 
#### Max porosity
label7f = tk.Label(seventh_frame, text="Max. DE")
label7f.place(x=370, y=140)

slider7f = customtkinter.CTkSlider(master=seventh_frame, from_=0, to=100,command=slider_event7f, state="normal", button_color='DodgerBlue3', width= 70)
slider7f.place(x=460, y=140)

value7f= slider7f.get()
value7f= round(value7f, 0)

output_value7f = tk.StringVar(value=value7f)
output_field7f = tk.Entry(seventh_frame, textvariable=output_value7f, state='readonly', width = 10)
output_field7f.place(x=540,y=140)

#### step size
label7g = tk.Label(seventh_frame, text="Step size")
label7g.place(x=370, y=190)

slider7g = customtkinter.CTkSlider(master=seventh_frame, from_=1, to=50,command=slider_event7g, state="normal", button_color='DodgerBlue3', width= 70)
slider7g.place(x=460, y=190)

value7g= slider7g.get()
value7g= round(value7g, 1)

output_value7g = tk.StringVar(value=value7g) 
output_field7g = tk.Entry(seventh_frame, textvariable=output_value7g, state='readonly', width = 10)
output_field7g.place(x=540,y=190)

fig = Figure(figsize=(4.7, 2.7))
ax = fig.add_subplot(111)

canvas = FigureCanvasTkAgg(fig, master=seventh_frame)
canvas.draw()
canvas.get_tk_widget().place(x=800, y=10)

colorbar_created = False
colorbar = None

notebook.place(x=10, y=10,width=1366, height=1080)
root.mainloop()

C:\Users\meberle\AppData\Local\Temp\ipykernel_34132\2141579338.py:15: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\meberle\AppData\Local\anaconda3\envs\myenv_new\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\meberle\AppData\Local\Temp\ipykernel_34132\2141579338.py", line 130, in button7a_event
    model = keras.models.load_model('neural_network_model_II_20231011.h5')
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 